In [75]:
import io
import os
import re
import sys
import time
import numpy as np

import unicodedata

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

### Import Data + Preprocessing

In [4]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
    text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)

parent_dir = os.path.dirname(text_dir)

parent_dir

'/Users/spencerbraun/.keras/datasets'

In [11]:
def labeler(example, index):
    return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
    lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
    labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
    labeled_data_sets.append(labeled_dataset)
    
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
    all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

for ex in all_labeled_data.take(5):
    print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b"The weapon forth, and darkness veil'd his eyes.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Unshaken stood; and with like haughty mien,'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Stand and repulse the Trojan. Now be firm.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'with salt. When the meat was roasted, he set it on platters, and handed'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'As stands a pillar tall or towering oak,'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


### Vocab and Embeddings

In [15]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

In [17]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [20]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)
encoded_example = encoder.encode(example_text)
print(encoded_example)

b"The weapon forth, and darkness veil'd his eyes."
[14006, 10123, 1109, 4175, 3362, 9672, 11889, 14218, 8730]


In [70]:
char2idx = {u:i for i, u in enumerate(vocabulary_set)}
idx2char = np.array(vocabulary_set)

# text_as_int = np.array([char2idx[c] for c in text])

In [21]:
def encode(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy())
    return encoded_text, label

In [23]:
def encode_map_fn(text, label):
    # py_func doesn't set the shape of the returned tensors.
    encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

    # `tf.data.Datasets` work best if all components have a shape set
    #  so set the shapes manually: 
    encoded_text.set_shape([None])
    label.set_shape([])

    return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

# You want to use Dataset.map to apply this function to each element of the dataset. Dataset.map runs in graph mode.

#     Graph tensors do not have a value.
#     In graph mode you can only use TensorFlow Ops and functions.

# So you can't .map this function directly: You need to wrap it in a tf.py_function. The tf.py_function will pass regular tensors (with a value and a .numpy() method to access it), to the wrapped python function.


In [73]:
all_encoded_data

<MapDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>

In [45]:
for element in all_encoded_data: 
    print(element) 
    break


(<tf.Tensor: shape=(9,), dtype=int64, numpy=array([14006, 10123,  1109,  4175,  3362,  9672, 11889, 14218,  8730])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


### Split Data

In [51]:
for element in train_data:
    print(train_data)
    break

<ShuffleDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>


In [53]:
## have to pay attention to padded shapes - see above for shape\
## padding adds 0's to tokenize sentences so all data the same shape

train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=((None,), ()))


test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=((None,), ()))

In [54]:
## since we added 0
vocab_size += 1



### Model Build (Bidirectional LSTM)

In [55]:
model = tf.keras.Sequential()

In [56]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

In [57]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [58]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))

In [59]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [60]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
697/697 [==============================] - 87s 124ms/step - loss: 0.5316 - accuracy: 0.7387 - val_loss: 0.3886 - val_accuracy: 0.8242
Epoch 2/3
697/697 [==============================] - 81s 116ms/step - loss: 0.3031 - accuracy: 0.8674 - val_loss: 0.3577 - val_accuracy: 0.8364
Epoch 3/3
697/697 [==============================] - 61s 88ms/step - loss: 0.2307 - accuracy: 0.8994 - val_loss: 0.3811 - val_accuracy: 0.8384


### Model Build (Encoder-Decoder GRU RNN)

In [ ]:
model = tf.keras.Sequential()

In [ ]:
# Add an Embedding layer expecting input vocab size, and
# output embedding dimension of size 64.

model.add(tf.keras.layers.Embedding(vocab_size, 64))

In [ ]:
# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(layers.GRU(256, return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(layers.SimpleRNN(128))

model.add(layers.Dense(10))

model.summary() 

In [76]:

encoder_vocab = vocab_size
decoder_vocab = vocab_size

encoder_input = layers.Input(shape=(None, ))
encoder_embedded = layers.Embedding(input_dim=encoder_vocab, output_dim=64)(encoder_input)

# Return states in addition to output
output, state_h, state_c = layers.LSTM(
    64, return_state=True, name='encoder')(encoder_embedded)
encoder_state = [state_h, state_c]

decoder_input = layers.Input(shape=(None, ))
decoder_embedded = layers.Embedding(input_dim=decoder_vocab, output_dim=64)(decoder_input)

# Pass the 2 states to a new LSTM layer, as initial state
decoder_output = layers.LSTM(
    64, name='decoder')(decoder_embedded, initial_state=encoder_state)
output = layers.Dense(10)(decoder_output)

model = tf.keras.Model([encoder_input, decoder_input], output)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     1099456     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 64)     1099456     input_2[0][0]                    
______________________________________________________________________________________________

### Model Build (Keras Based not from docs)
See http://alexadam.ca/ml/2017/05/05/keras-vae.html

### Training

In [77]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [79]:
model.fit(train_data, epochs=3, validation_data=test_data)

In [ ]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

### Saving Outputs

In [ ]:
model.save('path_to_my_model.h5')

# Recreate the exact same model purely from the file
new_model = keras.models.load_model('path_to_my_model.h5')


In [ ]:

# Export the model to a SavedModel
model.save('path_to_saved_model', save_format='tf')

# Recreate the exact same model
new_model = keras.models.load_model('path_to_saved_model')

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Note that the optimizer state is preserved as well:
# you can resume training where you left off.
